In [168]:
import json
import os
import statistics

import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None

# Example

In [ ]:
scenarios_dir = '../../data/processed/scenarios/'
scenario_dir = 'scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0/'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [12]:
data = pd.read_csv(os.path.join(scenario_path, 'travels_results_1.csv'))

In [18]:
data.head()

,Unnamed: 0,agent_id,start_region,start_place_type,dest_region,dest_place_type,travel_start_time,transport_mode,is_driver
0,0,0,228,dom,126,praca,1219,0,1.0
1,1,0,126,praca,228,dom,1229,0,1.0
2,2,1,19,dom,1,inne,977,1,NaN
3,3,1,1,inne,19,dom,987,1,NaN
4,4,2,116,dom,147,inne,397,2,NaN


# Automa

In [ ]:
# {
# 	‘komunikacja’: {
#         średnia: średnia liczba przejazdów komunikacją,
#         ‘std’: wartość
#     }
# 	‘auta_kierowca’: jw
# 	‘auta_nie_kierowca’: jw
# 	itd.
# }

In [213]:
scenarios_dir = '../../data/processed/scenarios/'

# 0 0 0
# scenario_dir = 'scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0/'
# out_file = 'ms_scenario_0_0_0.json'

# 0 0 3
# scenario_dir = 'scenarios_2_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_3/'
# out_file = 'ms_scenario_0_0_3.json'

# 3 3 0
# scenario_dir = 'scenarios_10_1_pub_trans_comfort_dist-up_0_3_pub_trans_punctuality_dist-up_0_3_household_cars_dist-down_0_0/'
# out_file = 'ms_scenario_3_3_0.json'

# 3 3 3
# scenario_dir = 'scenarios_11_1_pub_trans_comfort_dist-up_0_3_pub_trans_punctuality_dist-up_0_3_household_cars_dist-down_0_3/'
# out_file = 'ms_scenario_3_3_3.json'

# 0 0 15
# scenario_dir = 'scenarios_9_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_15/'
# out_file = 'ms_scenario_0_0_15.json'

# 15 15 0
# scenario_dir = 'scenarios_5_1_pub_trans_comfort_dist-up_0_15_pub_trans_punctuality_dist-up_0_15_household_cars_dist-down_0_0/'
# out_file = 'ms_scenario_15_15_0.json'

# 15 15 15
scenario_dir = 'scenarios_6_1_pub_trans_comfort_dist-up_0_15_pub_trans_punctuality_dist-up_0_15_household_cars_dist-down_0_15/'
out_file = 'ms_scenario_15_15_15.json'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [214]:
unique_transport_mode = [
    'car_driver',
    'car_passenger',
    'public',
    'pedestrian',
    'bicycle'
]

statistics = [
    'mean', 
    'std'
]


json_dict = {}

for tm in unique_transport_mode:
    json_dict[tm] = {}
    for stat in statistics:
        json_dict[tm][stat] = []

In [215]:
def transport_mode_count(df_travels, tm):
    if tm.startswith('car_'):
        if tm == 'car_driver':
            transport_mode = 0
            is_driver = 1.0
        else:
            transport_mode = 0
            is_driver = 0.0

        count = df_travels[
            (df_travels['transport_mode'] == transport_mode) &
            (df_travels['is_driver'] == is_driver)
        ].shape[0]
    else:
        if tm == 'public':
            transport_mode = 1
        elif tm == 'pedestrian':
            transport_mode = 2
        elif tm == 'bicycle':
            transport_mode = 3
        else:
            transport_mode = -1
        
        count = df_travels[
            df_travels['transport_mode'] == transport_mode
        ].shape[0]
    
    return count

In [216]:
simulations_files = [f for f in os.listdir(scenario_path) if f.startswith(('agents_results_', 'travels_results_'))]
num_simulations = int(len(simulations_files) / 2)

# for simulation in tqdm(range(1, 3)):
for simulation in tqdm(range(1, num_simulations+1)):
    travels_file = 'travels_results_' + str(simulation) + '.csv'
    df_travels = pd.read_csv(os.path.join(scenario_path, travels_file), index_col=0)

    travels_num = df_travels.shape[0]

    for tm in unique_transport_mode:
        count = transport_mode_count(df_travels, tm)
        percentage = count / travels_num * 100
        for stat in statistics:
            json_dict[tm][stat].append(percentage)

100%|██████████| 100/100 [01:14<00:00,  1.35it/s]


In [217]:
import statistics

for tm in tqdm(unique_transport_mode):
    json_dict[tm]['mean'] = statistics.mean(json_dict[tm]['mean'])
    json_dict[tm]['std'] = statistics.stdev(json_dict[tm]['std'])

100%|██████████| 5/5 [00:00<00:00, 1111.13it/s]


In [218]:
json_dict

{'car_driver': {'mean': 26.00142229864044, 'std': 0.0674108111290693},
 'car_passenger': {'mean': 9.129015768698288, 'std': 0.04144142048604447},
 'public': {'mean': 34.54015167185465, 'std': 0.05752514629895113},
 'pedestrian': {'mean': 20.967568407721974, 'std': 0.05572320705849282},
 'bicycle': {'mean': 9.361841853084652, 'std': 0.04843156366373326}}

In [219]:
out_dir = '../../data/processed/mc/scenarios'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [220]:
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(json_dict, f)

# Scenarios

## Scenario 0 0 0

In [131]:
scenarios_dir = '../../data/processed/scenarios/'
scenario_dir = 'scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0/'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [132]:
total_data = pd.DataFrame(columns=['transport_mode', 'total_count', 'modal_split'])

for simulation_file in tqdm(os.listdir(scenario_path)):
    if simulation_file.startswith('travels_results_'):
        data = pd.read_csv(os.path.join(scenario_path, simulation_file))

        transport_mode_vc = data['transport_mode'].value_counts().rename_axis('transport_mode').reset_index(name='total_count')
        transport_mode_vc['modal_split'] = transport_mode_vc['total_count'] / transport_mode_vc['total_count'].sum() * 100

        total_data = pd.concat([total_data, transport_mode_vc])


var_map_dict = {
    0: 'komunikacja samochodowa',
    1: 'komunikacja zbiorowa',
    2: 'pieszo',
    3: 'rower',
}

total_data.replace({'transport_mode': var_map_dict}, inplace=True)

total_data.reset_index(inplace=True)

100%|██████████| 100/100 [00:20<00:00,  4.85it/s]


In [133]:
list(total_data['transport_mode'].unique())

['komunikacja samochodowa', 'komunikacja zbiorowa', 'pieszo', 'rower']

In [134]:
transport_modes = list(total_data['transport_mode'].unique())

for transport_mode in transport_modes:
    print(transport_mode)
    print(total_data[
        total_data['transport_mode'] == transport_mode
    ]['modal_split'].std(axis=0))

komunikacja samochodowa
0.09007087180728351
komunikacja zbiorowa
0.09402402575265148
pieszo
0.0655132359694321
rower
0.04254365743866603


In [136]:
total_data_grouped = total_data.groupby(['transport_mode'])['total_count'].sum().reset_index()

total_data_0_0_0 = total_data_grouped
total_data_0_0_0['modal_split'] = total_data_0_0_0['total_count'] / total_data_0_0_0['total_count'].sum() * 100

total_data_0_0_0

,transport_mode,total_count,modal_split
0,komunikacja samochodowa,12362723,39.313819
1,komunikacja zbiorowa,9880738,31.421034
2,pieszo,6364737,20.240049
3,rower,2838055,9.025098


## Scenario 0.3 0.3 0

In [141]:
scenarios_dir = '../../data/processed/scenarios/'
scenario_dir = 'scenarios_10_1_pub_trans_comfort_dist-up_0_3_pub_trans_punctuality_dist-up_0_3_household_cars_dist-down_0_0/'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [142]:
total_data = pd.DataFrame(columns=['transport_mode', 'total_count', 'modal_split'])

for simulation_file in tqdm(os.listdir(scenario_path)):
    if simulation_file.startswith('travels_results_'):
        data = pd.read_csv(os.path.join(scenario_path, simulation_file))

        transport_mode_vc = data['transport_mode'].value_counts().rename_axis('transport_mode').reset_index(name='total_count')
        transport_mode_vc['modal_split'] = transport_mode_vc['total_count'] / transport_mode_vc['total_count'].sum() * 100

        total_data = pd.concat([total_data, transport_mode_vc])


var_map_dict = {
    0: 'komunikacja samochodowa',
    1: 'komunikacja zbiorowa',
    2: 'pieszo',
    3: 'rower',
}

total_data.replace({'transport_mode': var_map_dict}, inplace=True)

total_data.reset_index(inplace=True)

100%|██████████| 100/100 [00:20<00:00,  4.91it/s]


In [143]:
transport_modes = list(total_data['transport_mode'].unique())

for transport_mode in transport_modes:
    print(transport_mode)
    print(total_data[
        total_data['transport_mode'] == transport_mode
    ]['modal_split'].std(axis=0))

komunikacja samochodowa
0.08900713935566026
komunikacja zbiorowa
0.08166558252994376
pieszo
0.07062194552837282
rower
0.046044550296766296


In [144]:
total_data_grouped = total_data.groupby(['transport_mode'])['total_count'].sum().reset_index()

total_data_3_3_0 = total_data_grouped
total_data_3_3_0['modal_split'] = total_data_3_3_0['total_count'] / total_data_3_3_0['total_count'].sum() * 100

total_data_3_3_0

,transport_mode,total_count,modal_split
0,komunikacja samochodowa,11506820,36.607064
1,komunikacja zbiorowa,10733408,34.146580
2,pieszo,6444110,20.500881
3,rower,2748994,8.745474


## Scenario 0 0 0.3

In [145]:
scenarios_dir = '../../data/processed/scenarios/'
scenario_dir = 'scenarios_2_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_3/'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [146]:
total_data = pd.DataFrame(columns=['transport_mode', 'total_count', 'modal_split'])

for simulation_file in tqdm(os.listdir(scenario_path)):
    if simulation_file.startswith('travels_results_'):
        data = pd.read_csv(os.path.join(scenario_path, simulation_file))

        transport_mode_vc = data['transport_mode'].value_counts().rename_axis('transport_mode').reset_index(name='total_count')
        transport_mode_vc['modal_split'] = transport_mode_vc['total_count'] / transport_mode_vc['total_count'].sum() * 100

        total_data = pd.concat([total_data, transport_mode_vc])


var_map_dict = {
    0: 'komunikacja samochodowa',
    1: 'komunikacja zbiorowa',
    2: 'pieszo',
    3: 'rower',
}

total_data.replace({'transport_mode': var_map_dict}, inplace=True)

total_data.reset_index(inplace=True)

100%|██████████| 100/100 [00:20<00:00,  4.95it/s]


In [147]:
transport_modes = list(total_data['transport_mode'].unique())

for transport_mode in transport_modes:
    print(transport_mode)
    print(total_data[
        total_data['transport_mode'] == transport_mode
    ]['modal_split'].std(axis=0))

komunikacja zbiorowa
0.07688851904994906
komunikacja samochodowa
0.08437993544355891
pieszo
0.07222312904478517
rower
0.04307761010030198


In [148]:
total_data_grouped = total_data.groupby(['transport_mode'])['total_count'].sum().reset_index()

total_data_0_0_3 = total_data_grouped
total_data_0_0_3['modal_split'] = total_data_0_0_3['total_count'] / total_data_0_0_3['total_count'].sum() * 100

total_data_0_0_3

,transport_mode,total_count,modal_split
0,komunikacja samochodowa,10539424,33.523908
1,komunikacja zbiorowa,10926328,34.754577
2,pieszo,6773475,21.545139
3,rower,3199303,10.176376
